## TicTacToe

### Imports and Utils

In [1]:
"""
Importing the necessary libraries
"""
import numpy as np

# Remove all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
Utilities for the game
"""

n_board = 3

def board_encoding(board: np.ndarray) -> int:
    """
    board: np.ndarray: Playing board as an array

    Return: int: board encoding
    """

    encode = 0
    power = 1
    for row in range(n_board):
        for col in range(n_board):
            if board[row, col] == 1:
                encode += power
            elif board[row, col] == -1:
                encode += 2*power
            
            power *= n_board

    return encode

def print_board(board: np.ndarray):
    """
    board: np.ndarray: Playing board as an array
    """

    str_board = [
        ['.', '.', '.'],
        ['.', '.', '.'],
        ['.', '.', '.'],
    ]
    for row in range(n_board):
        for col in range(n_board):
            if board[row, col] == 1:
                str_board[row][col] = 'X'
            elif board[row, col] == -1:
                str_board[row][col] = 'O'

    for row in range(n_board):
        print('  '.join(str_board[row]))

def is_full(board: np.ndarray) -> bool:
    """
    board: np.ndarray: Playing board as an array

    Return: bool: if the board is full
    """

    for row in range(n_board):
        for col in range(n_board):
            if board[row, col] == 0:
                return False
    return True

def has_won(board: np.ndarray, player: int) -> bool:
    """
    board: np.ndarray: Playing board as an array
    player: int: Numeric encoding

    Return: bool: if player has won
    """

    # check for horizontal
    for row in range(n_board):
        for col in range(n_board):
            if board[row, col] != player:
                break
        else:
            return True
        
    # check for vertical
    for col in range(n_board):
        for row in range(n_board):
            if board[row, col] != player:
                break
        else:
            return True
        
    # check for diagonal 1
    for row in range(n_board):
        if board[row, row] != player:
            break
    else:
        return True
    
    # check for diagonal 2
    for row in range(n_board):
        if board[row, n_board - row - 1] != player:
            return False
    else:
        return True

### Creating Game Algorithm

In [3]:
class TicTacToe():
    """
    An Optimal TicTacToe player
    """

    def __init__(self, board: np.ndarray, player: int):
        """
        board: np.ndarray: Playing board as an array
        player: int: Numeric encoding
        """

        self.board = board
        self.player = player
        self.computed_moves = np.zeros((19681, 4), dtype=np.int32)

    def best_move(self, board: np.ndarray, player: int) -> np.ndarray:
        """
        board: np.ndarray: Playing board as an array
        player: int: Numeric encoding
        """

        assert ~is_full(board)
        assert ~has_won(board, player)
        assert ~has_won(board, -1*player)

        no_candidate = 1
        candidate = np.zeros((1, 4), dtype=np.int32)

        encode = board_encoding(board)
        if self.computed_moves[encode][0]:
            return self.computed_moves[encode]
        
        for row in range(n_board):
            for col in range(n_board):
                if board[row, col] == 0:
                    board[row, col] = player
                    if has_won(board, player):
                        board[row, col] = 0
                        candidate = np.array([1, row, col, 1])
                        self.computed_moves[encode] = candidate
                        return candidate
                    board[row, col] = 0

        for row in range(n_board):
            for col in range(n_board):
                if board[row, col] == 0:
                    board[row, col] = player
                    if is_full(board):
                        board[row, col] = 0
                        candidate = np.array([1, row, col, 0])
                        self.computed_moves[encode] = candidate
                        return candidate
                    
                    response = self.best_move(board, -1*player)
                    board[row, col] = 0
                    if response[3] == -1:
                        candidate = np.array([1, row, col, 1])
                        self.computed_moves[encode] = candidate
                        return candidate
                    elif response[3] == 0:
                        candidate = np.array([1, row, col, 0])
                        self.computed_moves[encode] = candidate
                        no_candidate = 0
                    elif no_candidate:
                        candidate = np.array([1, row, col, -1])
                        self.computed_moves[encode] = candidate
                        no_candidate = 0
        
        self.computed_moves[encode] = candidate
        return candidate

### Defining Game System

In [8]:
board = np.zeros((3, 3))
current = 1
player = int(input("Choose Your Symbol: "))
computer = TicTacToe(board, -1*player)

while True:
    print_board(board)
    print("\n------------------------------------------------------------\n")
    if current == player:
        move = int(input("Enter move: "))
        row = int(move / n_board)
        col = int(move % n_board)
        assert board[row, col] == 0
        board[row, col] = player
    else:
        response = computer.best_move(board, computer.player)
        board[response[1], response[2]] = computer.player

    if has_won(board, current):
        print_board(board)
        print(f"\nPlayer {current} has won!")
        break
    elif is_full(board):
        print_board(board)
        print("\nIt's a Draw")
        break

    current *= -1
    

.  .  .
.  .  .
.  .  .

------------------------------------------------------------

.  .  .
.  .  .
.  .  X

------------------------------------------------------------

.  .  .
.  O  .
.  .  X

------------------------------------------------------------

.  .  .
.  O  .
.  X  X

------------------------------------------------------------

.  .  .
.  O  .
O  X  X

------------------------------------------------------------

.  .  X
.  O  .
O  X  X

------------------------------------------------------------

.  .  X
.  O  O
O  X  X

------------------------------------------------------------

.  .  X
X  O  O
O  X  X

------------------------------------------------------------



ValueError: invalid literal for int() with base 10: ''